In [48]:
import pandas as pd
import xlrd

filepath = '1999_referenda/results_by_polling_place/v4_3ADEL.XLS'

In [75]:
x = xlrd.open_workbook(filepath)
x_sheet = x.sheet_by_index(0)

date = x_sheet.cell(0,0).value.strip()
referenda = x_sheet.cell(5,0).value.strip()
state = x_sheet.cell(8,0).value.strip()
electorate = x_sheet.cell(9,0).value.strip()

In [63]:
#electorate contains name and enrollment in the form \"Adelaide (86272)\"

needle = electorate.find('(')
enrolment = electorate[needle:].strip() # take right of needle
enrolment = enrolment[1:-1] # remove first and last chars, ie, remove brackets

electorate = electorate[:needle-1].strip() # take everything from the start to one place before the bracket

print(enrolment)
print(electorate)

86272
Adelaide


In [91]:
def state_abbreviation(state):
    spaces = state.count(' ')
    
    if spaces == 2:
        bits = state.split(' ')
        r=''
        for b in bits:
            r = r + b[:1].upper() # for each word in state grab first letter
        return r
    elif 'Australia' in state:
        r = state[:1].upper() + 'A'
        return r
    elif state == 'Queensland':
        return 'QLD'
    elif state == 'Northern Territory':
        
    else:
        r = state[:3].upper()
        return r
print(state_abbreviation('New South Wales'))
print(state_abbreviation('Victoria'))
print(state_abbreviation('Queensland'))
print(state_abbreviation('South Australia'))
print(state_abbreviation('Western Australia'))
print(state_abbreviation('Tasmania'))
print(state_abbreviation('Northern Territory'))
print(state_abbreviation('Australian Capital Territory'))

NSW
VIC
QUE
SA
WA
TAS
NOR
ACT


In [76]:
# date in the form "Referendum: 6 November 1999"
needle = date.find(':')
needle = needle + 1 # skip the semicolon and the space
date = date[needle:].strip()

# parse as date
date = pd.to_datetime(date)
print(date)

6 November 1999
1999-11-06 00:00:00


In [46]:
columnheaders = ['polling_place_raw',
'yes_or_no',
'yes_n',
'yes_p',
'no_n',
'no_p',
'formal_n',
'formal_p',
'informal_n',
'informal_p',
'total_n',
'total_p']

df_result = pd.read_excel(
    filepath,
    skiprows = 13,
    names=columnheaders
)

# fix percentages
lambda_fix_percentages = lambda x: x/100

# run on the relevant columns
df_result['yes_p'] = df_result['yes_p'].apply(lambda_fix_percentages)
df_result['no_p'] = df_result['no_p'].apply(lambda_fix_percentages)
df_result['formal_p'] = df_result['formal_p'].apply(lambda_fix_percentages)
df_result['informal_p'] = df_result['informal_p'].apply(lambda_fix_percentages)
df_result['total_p'] = df_result['total_p'].apply(lambda_fix_percentages)

# if bracket in string, return left of bracket, if not, return whole string
def left_of_bracket(s):
    if '(' in s:
        needle = s.find('(')
        r = s[:needle-1].strip()
        return r
    else:
        return s

lambda_polling_places = lambda x: left_of_bracket(x)

# use above function to make a polling place column, i.e, remove ' (Adelaide)' from 'Enfield North (Adelaide)'
df_result['polling_place'] = df_result['polling_place_raw'].apply(lambda_polling_places)

print(df_result)

               polling_place_raw yes_or_no  yes_n   yes_p   no_n    no_p  \
0                  Adelaide East       Yes    465  0.6700    229  0.3300   
1              Adelaide Hospital       Yes    187  0.6172    116  0.3828   
2                 Adelaide South       Yes   1056  0.6467    577  0.3533   
3                  Adelaide West       Yes    205  0.6487    111  0.3513   
4              Black Forest East       Yes   1206  0.6009    801  0.3991   
5              Blair Athol North        No    628  0.4102    903  0.5898   
6                      Broadview        No    971  0.4793   1055  0.5207   
7                       Brompton       Yes    581  0.6109    370  0.3891   
8                  Clarence Park       Yes    661  0.5669    505  0.4331   
9                   College Park       Yes    765  0.6722    373  0.3278   
10            Croydon (Adelaide)       Yes    256  0.5482    211  0.4518   
11                  Croydon Park        No    819  0.4881    859  0.5119   
12  Croydon 